In [1]:
!pip install -U langchain-community
!pip install faiss-cpu
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.5 MB/s eta 0:00:00


In [2]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

In [3]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [4]:
# 初始化嵌入模型
embeddings = OpenAIEmbeddings()

<ipython-input-4-b0fd89d06ed4>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [5]:
# 创建商品列表（模拟文档）
products = [
    Document(page_content="这款智能手机拥有强大的处理器和高清摄像头。"),
    Document(page_content="这款笔记本电脑轻薄便携，适合商务旅行。"),
    Document(page_content="这款智能手表支持健康监测和消息提醒。")
]

In [6]:
# 创建向量存储
vectorstore = FAISS.from_documents(products, embeddings)

# 一、采用默认的相似度进行检索

In [7]:
# 查询向量存储
query = "最好的智能手机"
docs = vectorstore.similarity_search(query, k=2)

In [8]:
# 打印结果
for doc in docs:
    print(doc.page_content)

这款智能手机拥有强大的处理器和高清摄像头。
这款智能手表支持健康监测和消息提醒。


# 二、最大边际MMR检索
默认情况下，向量存储检索器使用相似性搜索。如果底层向量存储支持最大边际相关性搜索，可以将其指定为搜索类型。

In [9]:
retriever = vectorstore.as_retriever(search_type="mmr")
docs = retriever.get_relevant_documents("最好的智能手机")
# 打印结果
for doc in docs:
    print(doc.page_content)

<ipython-input-9-edb2c3d7a187>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("最好的智能手机")


这款智能手机拥有强大的处理器和高清摄像头。
这款智能手表支持健康监测和消息提醒。
这款笔记本电脑轻薄便携，适合商务旅行。


# 三、相似度阈值检索
还可以设置一种检索方法，该方法设置相似度分数阈值，并且仅返回分数高于该阈值的文档。

In [10]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)
docs = retriever.get_relevant_documents("最好的智能手机")
# 打印结果
for doc in docs:
    print(doc.page_content)

这款智能手机拥有强大的处理器和高清摄像头。
这款智能手表支持健康监测和消息提醒。
这款笔记本电脑轻薄便携，适合商务旅行。


# 四、指定前k个
还可以指定在检索时使用的搜索 kwargs，例如 k。

In [11]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
docs = retriever.get_relevant_documents("最好的智能手机")
len(docs)

1

In [12]:
# 打印结果
for doc in docs:
    print(doc.page_content)

这款智能手机拥有强大的处理器和高清摄像头。
